<a href="https://colab.research.google.com/github/Snowdenstyll/keno/blob/main/jupyterNotebooks/Keno_Lottery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [66]:
df = pd.read_csv("2024.csv")

In [67]:
df.head()
df.tail()
df.info()
df.describe()
df.drop(['PlayDate', 'AP'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   PlayDate  141 non-null    object
 1   AP        141 non-null    object
 2   N01       141 non-null    int64 
 3   N02       141 non-null    int64 
 4   N03       141 non-null    int64 
 5   N04       141 non-null    int64 
 6   N05       141 non-null    int64 
 7   N06       141 non-null    int64 
 8   N07       141 non-null    int64 
 9   N08       141 non-null    int64 
 10  N09       141 non-null    int64 
 11  N10       141 non-null    int64 
 12  N11       141 non-null    int64 
 13  N12       141 non-null    int64 
 14  N13       141 non-null    int64 
 15  N14       141 non-null    int64 
 16  N15       141 non-null    int64 
 17  N16       141 non-null    int64 
 18  N17       141 non-null    int64 
 19  N18       141 non-null    int64 
 20  N19       141 non-null    int64 
 21  N20       141 no

In [68]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [69]:
# All our games
number_of_rows = df.values.shape[0]
# Amount of games we need to take into consideration for prediction
window_length = 10
number_of_features = df.values.shape[1]

X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)


In [70]:
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [71]:
# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
#batch_size = 40
batch_size = 70

# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(70))
# Adding the last output layer
model.add(Dense(number_of_features))

In [72]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.95, random_state=95)

# Train the model
history = model.fit(X, y, batch_size=batch_size, epochs=1000, verbose=2)
#history = model.fit(X_train, y_train, batch_size=100, epochs=600, verbose=2)

Epoch 1/1000
2/2 - 17s - loss: 1.0168 - accuracy: 0.0229 - 17s/epoch - 8s/step
Epoch 2/1000
2/2 - 0s - loss: 1.0131 - accuracy: 0.0611 - 56ms/epoch - 28ms/step
Epoch 3/1000
2/2 - 0s - loss: 1.0095 - accuracy: 0.0992 - 54ms/epoch - 27ms/step
Epoch 4/1000
2/2 - 0s - loss: 1.0065 - accuracy: 0.0382 - 55ms/epoch - 28ms/step
Epoch 5/1000
2/2 - 0s - loss: 1.0021 - accuracy: 0.0611 - 51ms/epoch - 26ms/step
Epoch 6/1000
2/2 - 0s - loss: 0.9985 - accuracy: 0.1069 - 40ms/epoch - 20ms/step
Epoch 7/1000
2/2 - 0s - loss: 0.9961 - accuracy: 0.1069 - 40ms/epoch - 20ms/step
Epoch 8/1000
2/2 - 0s - loss: 0.9917 - accuracy: 0.0611 - 43ms/epoch - 21ms/step
Epoch 9/1000
2/2 - 0s - loss: 0.9870 - accuracy: 0.0534 - 39ms/epoch - 19ms/step
Epoch 10/1000
2/2 - 0s - loss: 0.9831 - accuracy: 0.0763 - 35ms/epoch - 17ms/step
Epoch 11/1000
2/2 - 0s - loss: 0.9786 - accuracy: 0.0611 - 34ms/epoch - 17ms/step
Epoch 12/1000
2/2 - 0s - loss: 0.9727 - accuracy: 0.0840 - 36ms/epoch - 18ms/step
Epoch 13/1000
2/2 - 0s - lo

In [73]:
loss, accuracy = model.evaluate(X, y, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

5/5 [==============================] - 3s 10ms/step - loss: 0.0029 - accuracy: 0.9542
Test Loss: 0.002852250821888447, Test Accuracy: 0.9541984796524048
4/4 [==============================] - 0s 10ms/step - loss: 0.0029 - accuracy: 0.9600
Test Loss: 0.002866991562768817, Test Accuracy: 0.9599999785423279


In [74]:
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)

In [75]:
import datetime
import pytz

# Get the current time in a specific timezone
tz = pytz.timezone('America/New_York')
current_day = datetime.datetime.now(tz)

time_label = "Midday" if (current_day.hour <= 14) else "Evening"

label = current_day.strftime("%Y_%m_%d_") + time_label
model.save(f'Models/{label}.keras')
print("saved model")

saved model


In [76]:
df1 = df.copy()
to_predict = df1.tail(window_length)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict

<ipython-input-76-efb52fef0daf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)


,N01,N02,N03,N04,N05,N06,N07,N08,N09,N10,N11,N12,N13,N14,N15,N16,N17,N18,N19,N20
131,4,12,24,26,29,31,33,39,40,42,50,51,52,53,54,58,59,63,68,70
132,1,4,8,9,15,16,21,26,28,30,38,39,43,47,48,55,61,63,64,66
133,2,12,13,14,17,21,25,28,29,30,34,40,47,56,58,64,65,66,67,68
134,4,9,10,11,13,16,18,21,22,34,35,36,39,44,49,50,53,58,60,66
135,2,3,7,8,10,11,13,22,24,25,31,37,38,39,46,49,55,58,59,62
136,2,4,6,9,11,13,19,23,26,30,35,36,42,44,50,55,58,59,64,66
137,8,10,12,13,20,21,26,31,34,35,37,45,46,52,54,58,62,65,68,69
138,1,4,7,9,10,21,33,36,37,38,40,42,47,51,53,57,61,64,65,70
139,1,8,11,20,23,25,28,29,30,31,39,42,45,47,48,53,55,58,62,63


In [84]:
prediction = np.array(df.tail(1))

x_next = scaler.transform(to_predict)
y_pred = model.predict(np.array([x_next]))

temp = np.array(np.unique(scaler.inverse_transform(y_pred).astype(int)[0]))
combined_array = np.concatenate((temp, np.array(prediction[0])))
winning_numbers = np.bincount(combined_array)
winners = np.unique(combined_array)

uniques, uniq_idx, counts = np.unique(combined_array,return_index=True,return_counts=True)
duplicates = combined_array[uniq_idx[counts>=2] ]  # <--- Get duplicates

top_5_indices = np.argsort(y_pred[0], axis=0)[::-1][:-10]
top_5_numbers = []

winning_indices = np.where(np.isin(prediction[0], duplicates))[0]

# Print the winning indices
print("The winning indices:", winning_indices)

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices:", top_5_indices)
print("The top 5 numbers:", top_5_numbers)

print("The predicted numbers for the lottery game are:  ", temp)
print("The actual numbers in the last lottery game were:", prediction[0])
print("\n")
print("winners:", duplicates)
print(len(duplicates)/20)


1/1 [==============================] - 0s 61ms/step
The winning indices: [ 5  6  8 10 12 14 15 18]
The top 5 indices: [ 3  2  4  5  1  6  7 15 10 16]
The top 5 numbers: [22, 21, 23, 25, 12, 28, 30, 57, 40, 58]
The predicted numbers for the lottery game are:   [ 3 11 18 24 25 26 28 31 33 40 43 46 48 51 57 60 61 62 64]
The actual numbers in the last lottery game were: [ 5 12 21 22 23 25 28 30 31 32 40 41 46 47 48 57 58 59 60 63]


winners: [25 28 31 40 46 48 57 60]
0.4


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


# Predict the next set of numbers

In [81]:
next = df.copy()
next = next.tail(window_length)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))
prediction = scaler.inverse_transform(y_next_pred).astype(int)
print("The predicted numbers for the lottery game:", temp)
print("\n")
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]
top_5_numbers = []
for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)

top_10_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:10]
top_10_numbers = []
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)
print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)


1/1 [==============================] - 0s 24ms/step
The predicted numbers for the lottery game: [ 3 11 18 24 25 26 28 31 33 40 43 46 48 51 57 60 61 62 64]


The top 5 indices [16  3 15 17  4]
The top 5 numbers: [62, 18, 59, 63, 21]


The top 10 indices [16  3 15 17  4 13 14 12  6 11]
The top 10 numbers: [62, 18, 59, 63, 21, 51, 53, 47, 26, 43]


winning indices [ 5  6  8 10 12 14 15 18]
winning numbers [22, 26, 30, 39, 47, 53, 59, 64]


In [85]:
next = df.copy()
next = next.tail(1)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))

# Get the actual values of the top 10 predictions
prediction = scaler.inverse_transform(y_next_pred).astype(int)

# Get the indices of the top 10 predictions (change axis=0 if predictions are along a different axis)
top_10_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:10]
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]

top_10_numbers = []
top_5_numbers = []

# index [0,2,4] usually are correct
# Loop through each index in top_10_indices
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_5_numbers.append(number)

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)


print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)
print("\n")

print("The predicted numbers:", prediction)


1/1 [==============================] - 0s 22ms/step
The top 5 indices [17  8 15 16  0]
The top 5 numbers: [60, 30, 53, 56, 3]


The top 10 indices [17  8 15 16  0  7  6 14  1 18]
The top 10 numbers: [60, 30, 53, 56, 3, 26, 23, 49, 6, 63]


winning indices [ 5  6  8 10 12 14 15 18]
winning numbers [19, 23, 30, 36, 43, 49, 53, 63]


The predicted numbers: [[ 3  6  9 13 16 19 23 26 30 33 36 39 43 46 49 53 56 60 63 67]]
